# Generative process

## Definitions

We can generate data according to EBM. 

$S \sim {\rm UniformPermutation}(\cdot)$

$S$ follows a distribution of uniform permutation. That means the ordering of biomarkers is random. 

$k_j \sim {\rm DiscreteUniform}(N)$

$k_j$ follows a discrete uniform distribution, which means a participant is equally likely to fall in a progression stage (e.g., from $0$ to $4$, where $0$ indicate this participant is healthy.)

$X_{S(n)j} | S, k_n  \sim I(z_j == 1) \left[ I(n \leq k_j ) p(X_{S(n)j} \mid \theta_{S(n)} ) +I(n \gt k_j) p(X_{S(n)j} \mid \phi_{S(n)}) \right] +  \left(1-I(z_j==1) \right) p(X_{S(n)j} \mid \phi_{S(n)})$

## Parameters

$S$ denotes the ordering of a sequence of biomarkers. 

$N$: number of observed biomarkers.

$n$: a specific biomarker; e.g., biomarker $b$.

$J$: number of participants. 

$j$ denotes a participant. 

$X$ is observed values of biomarkers; it is a matrix of dimension of $N \times J$ or $J \times N$.

$k$, a scalar whose value is the participant's stage of the disease

$K$: number of disease stages

$k_n$ means the disease stage that a specific biomarker $n$ indicates. 

$k_j$: disease stage that a participant is at. 

$X_{nj}$ means the observed value of the biomarker $n$ in participant $j$. 

$\theta_n$ is the parameters for the probability density function (PDF) of observed value of biomarker $n$ when this biomarker has been affected by the disease. Let's assume this distribution is a Gaussian distribution.

$\phi_n$ is the parameters for the probability density function (PDF) of observed value of biomarker $n$ when this biomarker has NOT been affected by the disease. 

## Simulation

**We are going to generate biomarker values for each participant by randomly drawing from distributions defined by $\theta$ or $\phi$**

In [1]:
import numpy as np
import scipy.stats as stats
import altair as alt
import pandas as pd

def simulate_ebm(N, J):
    """
    Simulate an Event-Based Model (EBM) for disease progression.
    
    Args:
    N (int): Number of biomarkers.
    J (int): Number of participants.
    
    Returns:
    tuple: A tuple containing:
        - S (numpy.ndarray): The random permutation representing the order of biomarker implication.
        - kjs (numpy.ndarray): The disease stages for participant.
        - X (numpy.ndarray): The biomarker matrix with participant data.
          - Each cell in X is tuple containing participantId, biomarker value, 
            disease stage of this participant, disease stage current biomarker indicates
            and healthy status
    """
    
    # Random permutation for ordering biomarkers, starting from 0
    # S indicates the disease progression, S[0] -> stage1, S[1] -> stage2
    S = np.random.permutation(N)
    
    # Generate a random stage for each participant
    # The stage should be between 0 and N, inclusive
    kjs = np.random.randint(0, N+1, size=J)
    
    # Initiate biomarker matrix (J participants x N biomarkers), with entries as None
    X = np.full((J, N), None, dtype=object)

    # Generate theta and phi, which are both a function of biomarker n
    # np.random.seed(42)
    theta_means = {n: np.random.randint(low=0, high=9) for n in range(N)}
    theta_vars = {n: np.random.rand() for n in range(N)}
    phi_means = {n: np.random.randint(low=0, high=9) for n in range(N)}
    phi_vars = {n: np.random.rand() for n in range(N)}
    theta = {n: stats.norm(theta_means[n], theta_vars[n]) for n in range(N)}
    phi = {n: stats.norm(phi_means[n], phi_vars[n]) for n in range(N)}
    
    # Iterate through participants
    for j in range(J):
        # Iterate through biomarkers
        for n in range(N):
            # Disease stage of the current participant
            k_j = kjs[j]
            # Disease stage indicated by the current biomarker
            # Note that biomarkers always indicate that the participant is diseased
            # Thus, k_n >= 1
            k_n = np.where(S == n)[0][0] + 1
            
            # Assign values based on whether the participant's stage is at or past the biomarker's stage
            if k_j >= 1:
                if k_j >= k_n:
                    X[j, n] = (j, theta[n].rvs(), k_j, k_n, 'diseased') 
                else:
                    X[j, n] = (j, phi[n].rvs(), k_j, k_n, 'healthy')  
            else:
                X[j, n] = (j, phi[n].rvs(), k_j, k_n, 'healthy')        
    return S, kjs, X, theta_means, theta_vars, phi_means, phi_vars

/Users/hongtaoh/anaconda3/envs/bayes/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
N = 10
J = 100
S, kjs, X, theta_means, theta_vars, phi_means, phi_vars = simulate_ebm(N, J)

In [3]:
theta_means, theta_vars, phi_means, phi_vars

({0: 2, 1: 0, 2: 0, 3: 5, 4: 0, 5: 3, 6: 3, 7: 5, 8: 8, 9: 5},
 {0: 0.6408355150507966,
  1: 0.057989880365859614,
  2: 0.7864164967476142,
  3: 0.21326966496051292,
  4: 0.1392323653038099,
  5: 0.7588234184986882,
  6: 0.7874262654370208,
  7: 0.9521866000198987,
  8: 0.901840006904379,
  9: 0.7769836109560627},
 {0: 3, 1: 6, 2: 7, 3: 5, 4: 1, 5: 5, 6: 6, 7: 6, 8: 1, 9: 1},
 {0: 0.5749781149320913,
  1: 0.31830338371658673,
  2: 0.37825556115180636,
  3: 0.36528355309120963,
  4: 0.5583050271544385,
  5: 0.1320948564959954,
  6: 0.6279284800047793,
  7: 0.21751281995201932,
  8: 0.18414666305042415,
  9: 0.5589523373088177})

In [4]:
df_means_vars = pd.DataFrame([theta_means, theta_vars, phi_means, phi_vars]).transpose()
df_means_vars.columns = ['theta_mean', 'theta_var', 'phi_mean', 'phi_var']
df_means_vars = df_means_vars.rename_axis("biomarker", axis=0).reset_index()
df_means_vars

,biomarker,theta_mean,theta_var,phi_mean,phi_var
0,0,2.0,0.640836,3.0,0.574978
1,1,0.0,0.057990,6.0,0.318303
2,2,0.0,0.786416,7.0,0.378256
3,3,5.0,0.213270,5.0,0.365284
4,4,0.0,0.139232,1.0,0.558305
5,5,3.0,0.758823,5.0,0.132095
6,6,3.0,0.787426,6.0,0.627928
7,7,5.0,0.952187,6.0,0.217513
8,8,8.0,0.901840,1.0,0.184147
9,9,5.0,0.776984,1.0,0.558952


In [5]:
df_means_vars.to_csv('data/means_vars.csv', index=False)

In [6]:
S

array([9, 6, 8, 2, 4, 5, 1, 7, 0, 3])

In [7]:
X[10]

array([(10, 2.64757115769906, 9, 9, 'diseased'),
       (10, 0.018829755354210265, 9, 7, 'diseased'),
       (10, -0.32668415734168393, 9, 4, 'diseased'),
       (10, 5.127265959221471, 9, 10, 'healthy'),
       (10, -0.14922320147161813, 9, 5, 'diseased'),
       (10, 3.969161200348228, 9, 6, 'diseased'),
       (10, 2.8563729032511223, 9, 2, 'diseased'),
       (10, 4.846621384328361, 9, 8, 'diseased'),
       (10, 7.208311907874189, 9, 3, 'diseased'),
       (10, 5.3786506359931865, 9, 1, 'diseased')], dtype=object)

## Visualizing simulated results

With the above data structure, we can visualize the following data:

- Distribution of all biomarker values by biomarker
- Distribution of all biomarker values when the participant is at a certain disease stage
- Comparing a certain biomarker data 
- A certain participant's data

### Distribution of all biomarker values by biomarker

In [8]:
df = pd.DataFrame(X, columns = [f"Biomarker {i}" for i in range(N)])

# make this dataframe wide to long 
df_long = df.melt(var_name = "Biomarker", value_name="Value")

# exapand the value column into a dataframe
values_df = df_long['Value'].apply(pd.Series)
values_df.columns = ['participant', 'measurement', 'k_j', 'k_n', 'drawn_from']

# join values_df with df_long
df_expanded = df_long.drop('Value', axis = 1).join(values_df)

alt.Chart(df_expanded).transform_density(
    'measurement',
    as_=['measurement', 'Density'],
    groupby=['Biomarker']
).mark_area().encode(
    x="measurement:Q",
    y="Density:Q",
    facet = alt.Facet(
        "Biomarker:N",
        columns = 5
    ),
    color=alt.Color(
        'Biomarker:N'
    )
).properties(
    width= 140,
    height = 200,
).properties(
    title='Biomarker data for all participants across all stages'
)

alt.Chart(...)

In [9]:
df_expanded.to_csv("data/participant_data.csv", index=False)
df_expanded.head()


,Biomarker,participant,measurement,k_j,k_n,drawn_from
0,Biomarker 0,0,3.492852,2,9,healthy
1,Biomarker 0,1,2.287075,3,9,healthy
2,Biomarker 0,2,2.428878,2,9,healthy
3,Biomarker 0,3,2.518235,8,9,healthy
4,Biomarker 0,4,3.370019,8,9,healthy


In [10]:
# get data for conjugate priors
# biomarker 1 & drawn from diseased groups
df_expanded[(
    df_expanded.Biomarker == "Biomarker 1") & (
        df_expanded.drawn_from == "diseased")].to_csv("data/conjugate_data.csv", index=False)

In [11]:
# df_expanded[df_expanded.k_j == 0]

### Distribution of all biomarker values when the participant is at a certain disease stage

In [12]:
# biomarker data when the participant is at stage 6
df_kj_6 = df_expanded[df_expanded.k_j == 6]
df_kj_6

alt.Chart(df_kj_6).transform_density(
    'measurement',
    as_=['measurement', 'Density'],
    groupby=['Biomarker']
).mark_area().encode(
    x="measurement:Q",
    y="Density:Q",
    facet = alt.Facet(
        "Biomarker:N",
        columns = 5
    ),
    color=alt.Color(
        'Biomarker:N'
    )
).properties(
    width= 140,
    height = 200,
).properties(
    title='Biomarker data when the participant is at stage six'
)

alt.Chart(...)

### Comparing a certain biomarker data 

In [13]:
# select only biomarker 2 
bio_2_data = df_expanded[df_expanded.Biomarker=='Biomarker 2'].drop(['k_j', 'k_n', 'Biomarker'], axis = 1)
# biomarker2 data, comparing from diseased and healthy groups
alt.Chart(bio_2_data).transform_density(
    'measurement',
    as_=['measurement', 'Density'],
    groupby=['drawn_from']
).mark_area().encode(
    x="measurement:Q",
    y="Density:Q",
    facet = alt.Facet(
        "drawn_from:N",
    ),
    color=alt.Color(
        'drawn_from:N'
    )
).properties(
    width= 240,
    height = 200,
).properties(
    title='Biomarker2 data, compring healthy group and diseased group'
)

alt.Chart(...)

### A certain participant's data

In [14]:
# participant 10
participant10_data = df_expanded[df_expanded.participant == 10]
alt.Chart(participant10_data).mark_bar().encode(
    x='Biomarker',
    y='measurement',
    color=alt.Color(
        'drawn_from:N'
    ),
    tooltip=['Biomarker', 'drawn_from', 'measurement']
).interactive().properties(
    title=f'Biomarker data for participant10 (k_j = {participant10_data.k_j.to_list()[0]})'
)

alt.Chart(...)